In [2]:
from IPython.core.display import display, HTML
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys

In [4]:
import pandas as pd
import numpy as np
import scipy

from sklearn.model_selection import train_test_split

import torch
import torch.utils.data as data_utils
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [5]:
sys.path.append('.')

In [6]:
import ngm

In [7]:
# Load Data Set
node_values = np.genfromtxt('./BlogCatalog-dataset/data/group-edges.csv', delimiter=',').astype(int)
node_values = np.sort(node_values, axis=0)

nodes = np.genfromtxt('./BlogCatalog-dataset/data/nodes.csv', delimiter=',').astype(int)
groups = np.genfromtxt('./BlogCatalog-dataset/data/groups.csv', delimiter=',').astype(int)
edges = np.genfromtxt('./BlogCatalog-dataset/data/edges.csv', delimiter=',').astype(int)

edges_df = pd.read_csv('./BlogCatalog-dataset/data/edges.csv', names=['node', 'node(2)'])

edges_df = pd.crosstab(edges_df['node'], edges_df['node(2)'])
idx = edges_df.columns.union(edges_df.index)
edges_df = edges_df.reindex(index = idx, columns=idx, fill_value=0)

In [8]:
node_values

array([[    1,     1],
       [    2,     1],
       [    3,     1],
       ...,
       [10311,    39],
       [10311,    39],
       [10312,    39]])

In [9]:
edges_df

,1,2,3,4,5,6,7,8,9,10,...,10303,10304,10305,10306,10307,10308,10309,10310,10311,10312
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10308,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
10309,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10310,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10311,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data = scipy.sparse.csr_matrix(edges_df.values)
node_values_df = pd.read_csv('./BlogCatalog-dataset/data/group-edges.csv', names=['node(2)', 'val'])
node_values_df.drop_duplicates(subset ="node(2)", keep = "last", inplace = True) 
node_values_df.sort_values('node(2)', inplace=True)
node_values_df = node_values_df.set_index('node(2)')
group_data = node_values_df.values
group_data = np.delete(group_data, 0, 1).reshape(-1)
node_values_df['is_val'] = (node_values_df['val'] == 1)
new_df = pd.merge(edges_df, node_values_df, left_index=True, right_index=True)
new_df['is_val'] = new_df['is_val'].astype(int)
new_df.to_csv('/tmp/data.csv')

X_train, X_test, y_train, y_test = train_test_split(new_df[new_df.columns[:-2]], new_df['is_val'], test_size=0.33, random_state=42)
X_train = torch.tensor(X_train.values).float()
y_train = torch.tensor(y_train.values).float()

X_test = torch.tensor(X_test.values).float()
y_test = torch.tensor(y_test.values).float()

In [11]:
class MyDataset(Dataset):
    def __init__(self, values, labels):
        super(MyDataset, self).__init__()
        self.values = values
        self.labels = labels

    def __len__(self):
        return len(self.values)  # number of samples in the dataset

    def __getitem__(self, index):
        return self.values[index], self.labels[index]

batch_size=10
dataset = MyDataset(X_train, y_train)

counts = pd.Series(y_train).value_counts()
weights = len(y_train) / (len(np.unique(y_train)) * counts)
print(weights)
y_train_weighted = np.array(y_train)
for i in counts.index:
    #print(weights.loc[i])
    y_train_weighted[y_train_weighted==i] = weights.loc[i]
sampler = torch.utils.data.sampler.WeightedRandomSampler(y_train_weighted, len(y_train_weighted))                     

dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler)

testset = MyDataset(X_test, y_test)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=True)

# Initialize NN

## Initialize Hyperparameters



0.0     0.502912
1.0    86.362500
dtype: float64


In [13]:
# TODO: Read this from data
input_size = 10312 # The image size = 1x10312 = 10312
hidden_size = 50
num_classes = 2
model = ngm.base.Net1(input_size, hidden_size, num_classes)
model

Net1(
  (fc1): Linear(in_features=10312, out_features=50, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=50, out_features=2, bias=True)
)

In [ ]:
kwargs = {
    'num_epochs': 5,         # The number of times entire dataset is trained
    'batch_size': 10,      # The size of input data took for one iteration
    'learning_rate':0.001, # The speed of convergence
    'train_test_split':0.8
}
ngm.base.train(model, dataloader, **kwargs)

In [ ]:


    correct = 0
    total = 0
    all_predicted = []
    all_labels = []
    for i, (images, labels) in enumerate(testloader):
        images = Variable(images.view(-1, 1*10312)) 
        outputs = model(images.float()) 
        _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
        total += labels.size(0)                    # Increment the total count
        correct += (predicted == labels).sum()     # Increment the correct count
        all_predicted += list(predicted)
        all_labels += (labels)

    print('Accuracy: %d %%' % (100 * correct / total))